In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn


In [2]:
df = pd.read_csv('train.csv')

In [1]:
featuretarget= df.drop(['row_id', 'time_id', "investment_id"], axis=1)
corr_matrix=featuretarget.corr()
a=corr_matrix["target"].sort_values()
b=abs(a)
c=b.sort_values()



NameError: name 'df' is not defined

In [6]:
c[100:300]

f_276    0.014485
f_236    0.014531
f_152    0.014582
f_298    0.014584
f_22     0.014629
           ...   
f_280    0.046870
f_265    0.047300
f_119    0.049404
f_250    0.055838
f_231    0.062049
Name: target, Length: 200, dtype: float64

In [5]:
np.save('featureimportance',c)

In [7]:
featurename=list(c.index[100:300])




In [8]:
np.save('less faetures mlp with little dropout/featureimportance',featurename)

In [9]:
feature = df[featurename]
y = df["target"]
id=df["investment_id"]

In [10]:
del df

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature=torch.from_numpy(feature.values)
id=torch.from_numpy(id.values)
y=torch.from_numpy(y.values)
class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to("cuda")
        self.y = y.squeeze().to("cuda")
        self.id = id.squeeze().to(torch.int64).to("cuda")
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_train=FinData(feature=feature,id=id,y=y)
DataLoader1=DataLoader(Dataset_train,batch_size=128,shuffle=True)
DataLoader2=DataLoader(Dataset_train,batch_size=1024,shuffle=True)
DataLoader3=DataLoader(Dataset_train,batch_size=512,shuffle=True)

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,32)
        self.linear1=nn.Linear(32,64)
        self.linear2=nn.Linear(64,64)
        self.linear3=nn.Linear(200,256)
        self.linear4=nn.Linear(256,256)
        self.linear5=nn.Linear(320,512)
        self.linear6=nn.Linear(512,128)
        self.linear7=nn.Linear(128,32)
        self.linear8=nn.Linear(32,1)
        self.dropout=nn.Dropout(p=0.1)
      

    def forward(self, feature, id):
        id=self.embedding(id).squeeze()
        id=self.linear1(id).squeeze()
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)
        id=self.dropout(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)


        feature=self.linear3(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.dropout(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)

        x=torch.cat((id,feature),1)
        x=self.linear5(x).squeeze()
        x=x*torch.sigmoid(x)
        x=self.linear6(x).squeeze()
        x=x*torch.sigmoid(x)

        x=self.linear7(x).squeeze()
        x=self.dropout(x)
        x=x*torch.sigmoid(x)

        x=self.linear8(x).squeeze()
        x=40*torch.sigmoid(x)-20
        return x.squeeze()

In [19]:
model1_1 = NeuralNetwork().to("cuda")
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to("cuda")
model1_2=model1_2.double()

model2_1 = NeuralNetwork().to("cuda")
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to("cuda")
model2_2=model2_2.double()


model3_1 = NeuralNetwork().to("cuda")
model3_1=model3_1.double()
model3_2 = NeuralNetwork().to("cuda")
model3_2=model3_2.double()

In [20]:
learning_rate = 1e-3
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=learning_rate,weight_decay=1e-3)
optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer2_2 = torch.optim.Adam(model2_2.parameters(), lr=learning_rate,weight_decay=1e-3)
optimizer3_1 = torch.optim.Adam(model3_1.parameters(), lr=learning_rate,weight_decay=1e-4)
optimizer3_2 = torch.optim.Adam(model3_2.parameters(), lr=learning_rate,weight_decay=1e-3)
loss_fn1=torch.nn.MSELoss()

In [21]:
from tqdm import tqdm

In [22]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = loss_fn1(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_2(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = -torch.corrcoef(torch.stack((pred,y)))[0,1]
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [23]:
for i in range(10):
    print("epoch{}/{}".format(i+1,10))
    train_1(DataLoader1,model1_1,optimizer1_1)
    train_2(DataLoader1,model1_2,optimizer1_2)
    train_1(DataLoader2,model2_1,optimizer2_1)
    train_2(DataLoader2,model2_2,optimizer2_2)
    train_1(DataLoader3,model3_1,optimizer3_1)
    train_2(DataLoader3,model3_2,optimizer3_2)

epoch1/10


100%|██████████| 6136/6136 [01:06<00:00, 92.54it/s]


epoch2/10


100%|██████████| 6136/6136 [01:15<00:00, 81.22it/s]


epoch3/10


...

100%|██████████| 6136/6136 [01:08<00:00, 89.10it/s]


epoch9/10


100%|██████████| 6136/6136 [01:08<00:00, 89.72it/s]


epoch10/10


100%|██████████| 6136/6136 [15:49<00:00,  6.46it/s] 


In [24]:
torch.save(model1_1, 'less faetures mlp with little dropout/model1_1.pth')
torch.save(model1_2, 'less faetures mlp with little dropout/model1_2.pth')
torch.save(model2_1, 'less faetures mlp with little dropout/model2_1.pth')
torch.save(model2_2, 'less faetures mlp with little dropout/model2_2.pth')
torch.save(model3_1, 'less faetures mlp with little dropout/model3_1.pth')
torch.save(model3_2, 'less faetures mlp with little dropout/model3_2.pth')